In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time

from tqdm import tqdm

In [2]:
import tb1025_utils
import pandas as pd

## Here we will extract all the articles links from https://www.tb1025.cn/ webpage
URL:  https://www.tb1025.cn/

Tabs not extracted:
https://www.tb1025.cn/


------------
-----------
------------
------------
------------

In [4]:
def loop_article_page(total_page, custom_url, key_code, base_url):
    """
    
    """
    return_file = {
        "Data": [],
        "message": "success",
        "response": 200
    }
    All_url_links = {}
    if total_page == 0:
        total_page = 1
    
    try:
        for i in tqdm(range(1, total_page+1)):
            if i==1:
                final_url = custom_url+ "index.html"
            else:
                final_url = custom_url+ "index_" + str(i) + ".html"
                
            found_url_links = tb1025_utils.extract_tb1025_page_article_links(final_url, base_url)
            key = key_code + str(i)
            All_url_links[key] = found_url_links
        return_file["Data"] = All_url_links
        return return_file
    
    except Exception as e:
        return_file["Data"] = All_url_links
        return_file["message"] = e
        return_file["response"] = 404
        return return_file

In [5]:
def check_error_in_links(All_url_link, page_code, print_each_error=False):
    """
    
    
    """

    error_counter = 0
    for page_id in range(1, len(All_url_link)):
        page_key = page_code + str(page_id)
        try:
            All_url_link.get(page_key)
            if  All_url_link.get(page_key)["Response"]!= 200:
                error_counter += 1
                if print_each_error:
                    print(page_key, All_url_link.get(page_key)["message"])
        except Exception as e:
            print(page_key, e)

    print(f"Total error in {page_code}: {error_counter}")

In [6]:
def save_json(path, file_name, data):
    """
    
    """
    with open(path+file_name, "w") as outfile:
        json.dump(data, outfile, indent=4, ensure_ascii=False)
        print(f"\n\nSuccessfully saved: {file_name}")

### all data

In [3]:
df = pd.read_csv("./data/All_cate.csv")
df.head(20)

,name_of_cate,original_url,Last_page_link,list_page_no
0,གསར་འགྱུར།,https://www.tb1025.cn/sj/,/sj/index_2.html,2
1,ཆོས་འབྱུང་།,https://www.tb1025.cn/lg/qijun/,/lg/qijun/index_16.html,16
2,བཀའ་འགྱུར།,https://www.tb1025.cn/gj/,/gj/index_7.html,7
3,བསྟན་འགྱུར།,https://www.tb1025.cn/dj/,/dj/index_6.html,6
4,ཚད་མ།,https://www.tb1025.cn/dn/canma/,/dn/canma/index_3.html,3
5,ཕར་ཕྱིན།,https://www.tb1025.cn/dn/paxing/,/dn/paxing/index_11.html,11
6,དབུ་མ།,https://www.tb1025.cn/dn/huma/,/dn/huma/index_2.html,2
7,སྔགས།,https://www.tb1025.cn/dn/ne/,/dn/ne/index_2.html,2
8,ཆོ་ག,https://www.tb1025.cn/qj/qiaoga/,/qj/qiaoga/index_2.html,2
9,ཞལ་འདོན།,https://www.tb1025.cn/qj/xiaduan/,/qj/xiaduan/index_8.html,8


In [14]:
# sample_df = df.iloc[0:1]
sample_df = df

## check those
- file_name_code
- base_url
- starting index for page.
   - if lowest is 0 then index is 0 else 1

In [11]:
file_name_code = "tb1025"
base_url = "https://www.tb1025.cn"

In [15]:
for index, row in sample_df.iterrows():
    # print(row)
    name_of_cate = row["name_of_cate"]
    original_url = row["original_url"]
    Last_page_link = row["Last_page_link"]
    list_page_no = row["list_page_no"]

    custom_url = original_url

    total_page = list_page_no
    article_tag = name_of_cate
    key_code = "Page " + article_tag + " "
    all_links = loop_article_page(total_page, custom_url, key_code, base_url)
    
    print(f"Total page in {article_tag}: {len(all_links['Data'])}")
    check_error_in_links(all_links['Data'], key_code, print_each_error=True)

    # Saving the final file
    path = "./data/links/"
    file_name = f"{file_name_code}_ALL_link_{article_tag}.json"
    save_json(path, file_name, all_links['Data'])

100%|██████████| 2/2 [00:01<00:00,  1.07it/s]


Total page in གསར་འགྱུར།: 2
Total error in Page གསར་འགྱུར། : 0


Successfully saved: tb1025_ALL_link_གསར་འགྱུར།.json


100%|██████████| 16/16 [00:15<00:00,  1.06it/s]


Total page in ཆོས་འབྱུང་།: 16
Total error in Page ཆོས་འབྱུང་། : 0


Successfully saved: tb1025_ALL_link_ཆོས་འབྱུང་།.json


100%|██████████| 7/7 [00:06<00:00,  1.07it/s]


...

Total page in བོད་ལྗོངས།: 1
Total error in Page བོད་ལྗོངས། : 0


Successfully saved: tb1025_ALL_link_བོད་ལྗོངས།.json


100%|██████████| 1/1 [00:00<00:00,  1.10it/s]


Total page in ནང་ལོགས།: 1
Total error in Page ནང་ལོགས། : 0


Successfully saved: tb1025_ALL_link_ནང་ལོགས།.json


100%|██████████| 1/1 [00:01<00:00,  1.01s/it]

Total page in ལྷ་ཁང་།: 1
Total error in Page ལྷ་ཁང་། : 0


Successfully saved: tb1025_ALL_link_ལྷ་ཁང་།.json
